Import the pynrrd package supplementary class. Load the image and mask.

In [2]:
import nrrd
from scripts.ImageSliceViewer3D import ImageSliceViewer3D as isv 

# paths to image and corresponding mask
im_path = 'data/images/LIDC-IDRI-0001.nrrd'
msk_path = 'data/masks/LIDC-IDRI-0001.nrrd'

im_V,im_d = nrrd.read(im_path)
msk_V,msk_d = nrrd.read(msk_path)

Use `skimage.measure`'s `label` method to locate the radiologist-defined mask. Recall that each pixel in the mask was agreed to be in the lesion with minimum 50% agreement between 4 radiologists.

**WILL NEED TO ADJUST SCRIPT FOR >1 LESION PER SCAN**

In [3]:
from skimage.measure import label,regionprops

# label the mask into connected regions
nodule_labels,num_labels = label(msk_V,return_num=True)
nodule_props = regionprops(nodule_labels)

# the number of nodules in nodule_props corresponds to the number of connected regions
print('There are {} nodule(s) in this image.'.format(len(nodule_props)))     

There are 1 nodule(s) in this image.


In [4]:
# isolate the centroid of the nodule -- this will be our marker for the watershed algorithm
nodule_props[0].centroid
centroid = [round(i) for i in nodule_props[0].centroid]
centroid

[367, 316, 90]

In [5]:
from skimage.segmentation import watershed

# watershed?

In [6]:
from scipy.ndimage import median_filter as median

# denoise image
denoised = median(im_V,5)

print(denoised.shape)
isv(denoised)

(512, 512, 133)


interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [12]:
from skimage.morphology import erosion, dilation, ball

# try some funky stuff
max_image = dilation(denoised,ball(2))
min_image = erosion(denoised,ball(2))

gradient_image = max_image - min_image

isv(gradient_image)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [15]:
from skimage.measure import label

threshold_gradient = gradient_image < 200
isv(threshold_gradient)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [16]:
markers = label(threshold_gradient)
isv(markers)

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [ ]:
# process the watershed
labels = watershed(gradient_image, markers)

In [ ]:
isv(labels)